# Raster data preparations

In this exercise, the raster data for the classification excercises is prepared. For machine learning tasks, the input raster data must follow these rules.

1) Data and labels images must have the same:
* Coordinate system
* Resolution / pixel size
* Data origin (min x and y), so that one pixel in both files covers exactly the same area.

2) Labels image must be a raster image of type Integer and 1 band. The class numbers start from 0 and grow to `number_of_classes - 1`. So for binary classification the labels image has values 0 and 1. For 4 class multiclass labels, values 0 to 3. If needed, vector labels need to be rasterized.

3) Data image must be of type Float with values between 0 and 1 (sometimes -1 to 1), may have several bands, usually has.

Here we prepare labels for all data image area, for pixel-wise machine learning, data could be prepared also for specific points/pixels only. See for example: [SYKE land use classification with LUCAS points](https://geohpc.readthedocs.io/en/latest/lessons/L3/03_LandCoverClassification_syke_Parallelization.html)

## Data sources

#### Labels:
Data sources: 
* Finnish Food Authority, [Agricultural parcels](https://www.paikkatietohakemisto.fi/geonetwork/srv/eng/catalog.search#/metadata/e4467d0b-51f9-41f2-bd7e-8ff48a2cb083). Data downloaded from [GeoPortti GeoCubes](https://vm0160.kaj.pouta.csc.fi/geocubes/). Agricultural parcels are originally vector data, but have been rasterized to GeoCubes.
* LUKE, [MVMI forest inventory, site main class](http://urn.fi/urn:nbn:fi:fd-849cabd2-0ae5-3455-b6f3-a39086cce33c). Data downloaded from [Paituli](https://paituli.csc.fi/).
* SYKE, [CORINE](https://www.paikkatietohakemisto.fi/geonetwork/srv/eng/catalog.search#/metadata/%7B0B4B2FAC-ADF1-43A1-A829-70F02BF0C0E5%7D). Data downloaded from [GeoPortti GeoCubes](https://vm0160.kaj.pouta.csc.fi/geocubes/).


#### Data image:
* FMI, [10-days Sentinel2 mosaic](https://ckan.ymparisto.fi/dataset/sentinel-2-satellite-image-mosaics-s2gm-sentinel-2-satelliittikuvamosaiikki-s2gm), original images from ESA.

The advantage of useing GeoCubes is that there all data is pre-processed so that it fits the requirements mentioned above. 

All datasets used for this exercise have already same coordinate system (20m). The `stackstac` package takes care of harmonizing data origin and resolution (20m).

For finding the files [Paituli STAC](https://paituli.csc.fi/stac.html) is used. For longer intro to STAC and its usege see [CSC STAC examples](https://github.com/csc-training/geocomputing/tree/master/python/STAC).

## Data processing results

The goal of this exercise is to have 8 raster files:
* 4 labels files and 4 data files with different extents
* Extents: deep learning training, shallow learnging training, validation, test.
* All files:
    * Coordinate system: Finnish ETRS-TM35FIN, EPSG:3067
    * Resolution: 20m

#### Labels

Multiclass classification raster: 
* 1 - forest from forest inventory data
* 2 - fields from agricultural parcels data
* 3 - water from CORINE land cover data
* 0 - everything else 

#### Data image

* Sentinel2 mosaic, we include data from 2 different dates (May and July), to have more data values. Final dataset has 8 bands based on bands: 2, 3, 4 and 8 on dates: 2021-05-11 and 2021-07-21, reflection values scaled to [0 ... 1]. The bands source data is: 
     *  'b02' / '2021-05-11'
     *  'b02' / '2021-07-21'
     *  'b03' / '2021-05-11'
     *  'b03' / '2021-07-21'
     *  'b04' / '2021-05-11'
     *  'b04' / '2021-07-21'
     *  'b08' / '2021-05-11'
     *  'b08' / '2021-07-21'

## Data processing main steps
* Set STAC specs, file names and bbox extents for different files
* Connect to STAC catalog and define function to fetch data via STAC

#### Labels

1) Fetch original datasets via Paituli STAC and explore them.
2) Create a new labels dataset and calculate class values based on the original datasets
3) Save data to 4 files with different extents.

#### Data image

1) Fetch Sentinel-2 11-day mosaics via Paituli STAC, including only required bands
3) Normalize the data values, the reflectance values are originally between 0 and 1, but for storage they have been multiplied with 10 000, so here the normalization is simple division with 10 000.
4) Stack data from different dates just as bands.
3) Save data to 4 files with different extents.

This workbook requires at least 6Gb memory. 1 core is enough, but xarray could use more, if working with bigger datasets.

## Imports and paths

In [ ]:
import os
import numpy as np

# Handling raster data
import rioxarray
import xarray

# Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import pandas as pd

# STAC search and data fetching
import pyproj
import pystac_client
import stackstac

# Bbox creation for plotting the extents
import geopandas as gpd
from shapely.geometry import Polygon

Set STAC specs for all datasets: collcetion_id, time-period and names of assets.

Use [STAC browser](https://radiantearth.github.io/stac-browser/#/external/paituli.csc.fi/geoserver/ogc/stac/v1?.language=en) to find suitable datasets and their specs: 
* The collection ID is available behind I-source button.
* Asset names, select a random item and see the left pane.
* Available time, see Temporal Extent in STAC browser, use STAC search or look from GeoCubes or Paituli services, which years are available.

In [ ]:
# Paituli STAC
stac_endpoint = "https://paituli.csc.fi/geoserver/ogc/stac/v1"

# Fields 
fields_stac_id = "land_parcels_at_geocubes"
fields_time = "2016-07-01"
fields_assets = ["20m"]

# CORINE
corine_stac_id = "corine_land_cover_at_geocubes"
corine_time = "2018-07-01"
corine_assets = ["20m"]

# Forest
forest_stac_id = "luke_vmi_paatyyppi_at_paituli" 
forest_time = "2023-07-01"
forest_assets = ["luke_vmi_paatyyppi_at_paituli_tiff"]

# Sentinel-2 11-day mosaic
sentinel2_stac_id = "sentinel_2_11_days_mosaics_at_fmi"
sentinel2_time1 = "2021-05-11"
sentinel2_time2 = "2021-07-21"
sentinel2_assets = ['b02', 'b03', 'b04', 'b08']

Set folder and file names.

In [ ]:
# Folders
base_folder = os.path.join(os.sep, 'scratch', 'project_2000599', os.environ.get('USER'), 'lumi-aif-fmi', 'day2', 'exercise3')
exercise_folder = os.path.join(base_folder) 
data_folder = os.path.join(base_folder,'data', 'raster')

data_shallow = os.path.join(data_folder, 'data_shallow.tif')
data_deep = os.path.join(data_folder, 'data_deep.tif')
data_test = os.path.join(data_folder, 'data_test.tif')
data_validation = os.path.join(data_folder, 'data_validation.tif')

labels_shallow = os.path.join(data_folder, 'labels_shallow.tif')
labels_deep = os.path.join(data_folder, 'labels_deep.tif')
labels_test = os.path.join(data_folder, 'labels_test.tif')
labels_validation = os.path.join(data_folder, 'labels_validation.tif')
data_test

In [ ]:
# Set working directory
os.chdir(exercise_folder)

Create new folder for prepared data, if it does not exist yet.

In [ ]:
if not os.path.isdir(data_folder):
    os.makedirs(data_folder)

### Set extent for different files

Define extents for training, validation and test data. The data is from Southern Finland.

In [ ]:
x_min_3067 = 250000
y_min_3067 = 6690000
x_max_3067 = x_min_3067 + 200000 
y_max_3067 = y_min_3067 + 60000
bbox_3067 = (x_min_3067, y_min_3067, x_max_3067, y_max_3067)

x_min_3067_test = x_min_3067
x_max_3067_test = x_min_3067_test + 20000
x_min_3067_deep = x_max_3067_test
x_max_3067_deep = x_max_3067 - 40000
x_min_3067_shallow = x_min_3067_deep 
x_max_3067_shallow = x_min_3067_shallow + 20000
x_min_3067_validation = x_max_3067_deep
x_max_3067_validation = x_max_3067
y_min_3067_shallow = y_min_3067 
y_max_3067_shallow = y_min_3067 + 20000

data_epsg = 3067
data_epsg_string = "EPSG:" + str(data_epsg)

Plot the different extents.

In [ ]:
x_point_list_shallow = [x_min_3067_shallow, x_min_3067_shallow, x_max_3067_shallow, x_max_3067_shallow, x_min_3067_shallow]
x_point_list_deep = [x_min_3067_deep, x_min_3067_deep, x_max_3067_deep, x_max_3067_deep, x_min_3067_deep]
x_point_list_validation = [x_min_3067_validation, x_min_3067_validation, x_max_3067_validation, x_max_3067_validation, x_min_3067_validation]
x_point_list_test = [x_min_3067_test, x_min_3067_test, x_max_3067_test, x_max_3067_test, x_min_3067_test]
y_point_list = [y_min_3067, y_max_3067, y_max_3067, y_min_3067, y_min_3067]
y_point_list_shallow = [y_min_3067_shallow, y_max_3067_shallow, y_max_3067_shallow, y_min_3067_shallow, y_min_3067_shallow]

polygon_shallow = Polygon(zip(x_point_list_shallow, y_point_list_shallow))
polygon_deep = Polygon(zip(x_point_list_deep, y_point_list))
polygon_test = Polygon(zip(x_point_list_test, y_point_list))
polygon_validation = Polygon(zip(x_point_list_validation, y_point_list))

#polygons = [polygon_test, polygon_shallow, polygon_deep]
d = {'label': ['deep', 'shallow', 'test', 'validation'], 'geometry': [polygon_deep, polygon_shallow, polygon_test, polygon_validation]}
polygons = gpd.GeoDataFrame(d, crs=data_epsg_string)  
polygons.explore(column='label')

## STAC set-up

Open STAC end-point, the queries are done to here.

In [ ]:
catalog = pystac_client.Client.open(stac_endpoint)

Function to convert bbox to WGS-84 coordinate system, required by STAC search

In [ ]:
def bbox_to_wgs(bbox, epsg_code):
    x_min, y_min, x_max, y_max = bbox    
    epsg_string = "EPSG:" + str(epsg_code)
    long_min, lat_min = pyproj.Proj(epsg_string)(x_min, y_min, inverse=True)
    long_max, lat_max  = pyproj.Proj(epsg_string)(x_max, y_max, inverse=True)
    bbox_wgs = [long_min, lat_min, long_max, lat_max]
    return bbox_wgs

Function to query STAC catalog with given collection ID, time period and bbox. Fetch the data of found items, for given assets. Resolution (20m) is fixed here, but could be different from original data.

In [ ]:
def fetch_stac_layer_with_bbox(collection_id, assets, time_period, bbox, epsg):

    bbox_wgs = bbox_to_wgs(bbox, epsg)
    #data_epsg_string = "EPSG:" + str(epsg)
    
    search = catalog.search(
        bbox=bbox_wgs,
        collections=[collection_id],
        datetime=time_period,
    )
    
    cube = stackstac.stack(
        items=search.item_collection(),
        bounds=bbox_3067, 
        assets=assets,
        resolution=20,
        xy_coords='center', 
        epsg=epsg
    ).squeeze()
    if "time" in cube.dims:
        cube = cube.max(dim='time') # Here we use only data with one timestamp, but geocubes data is split to mapsheets, so this joins the mapsheets to same timestamp.
    cube.compute()
    return cube
    

## Labels processing

Fetch data via STAC.

In [ ]:
forest = fetch_stac_layer_with_bbox(forest_stac_id, forest_assets, forest_time, bbox_3067, data_epsg)
forest

In [ ]:
fields = fetch_stac_layer_with_bbox(fields_stac_id, fields_assets, fields_time, bbox_3067, data_epsg)
fields

In [ ]:
corine = fetch_stac_layer_with_bbox(corine_stac_id, corine_assets, corine_time, bbox_3067, data_epsg)
corine

### Explore original labels data

Plot small part of the data image.

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(15, 4))
corine.isel(x=slice(1000,2000), y=slice(2000,3000)).plot(ax=ax[0])
ax[0].set_title("Corine Land Cover")
fields.isel(x=slice(1000,2000), y=slice(2000,3000)).plot(ax=ax[1])
ax[1].set_title("Fields")
forest.isel(x=slice(1000,2000), y=slice(2000,3000)).plot(ax=ax[2])
ax[2].set_title("Forest main type")

### Reclassify

Create a new dataset with classification assigned based on the 3 original rasters.

In [ ]:
labels = fields.copy()
#labels = labels.where(labels != 2, other=0)
labels = xarray.where(labels == 2, 0, labels) # Not-field to 0
labels = xarray.where(labels == 1, 2, labels) # Fields to class 2
labels = xarray.where(forest < 50, 1, labels) # Forest to class 1
labels = xarray.where(corine == 47, 3, labels) # Rivers, to class 3
labels = xarray.where(corine == 48, 3, labels) # Lakes, to class 3
labels = xarray.where(corine == 49, 3, labels) # Sea, to class 3

In [ ]:
classes = [0, 1, 2, 3]
colors  = ["gray", "forestgreen", "lightyellow", "lightblue"]
cmap = ListedColormap(colors)
class_bounds = np.array(classes + [classes[-1] + 1])   # e.g., [11,12,21,22,23,31,32]
norm = BoundaryNorm(class_bounds, len(colors))
labels.isel(x=slice(1000,2000), y=slice(2000,3000)).plot(cmap=cmap, norm=norm)

See number of pixels in each class.

In [ ]:
def get_class_stats(data):
    stats = np.unique(data, return_counts=True)
    df = pd.DataFrame({
        "value": stats[0],
        "count": stats[1]
    })
    return df

In [ ]:
labels_stats = get_class_stats(labels)
print("Labels classes:")
labels_stats

### Save labels files

Add coordinate system info, as `rioxarray` wants it.

In [ ]:
labels.rio.write_crs(data_epsg, inplace=True)

Save the file with 4 different extents for deep and shallow model training, validation and testing.

In [ ]:
def save_bbox_to_geotiff(data, minx, miny, maxx, maxy, filename):
    clipped = data.rio.clip_box(minx, miny, maxx, maxy)
    clipped.rio.to_raster(filename, tiled=True)

In [ ]:
# Deep learning, model training
save_bbox_to_geotiff(labels, x_min_3067_deep, y_min_3067, x_max_3067_deep, y_max_3067, labels_deep)
# Shallow learning, model training
save_bbox_to_geotiff(labels, x_min_3067_shallow, y_min_3067_shallow, x_max_3067_shallow, y_max_3067_shallow, labels_shallow)
# Validation
save_bbox_to_geotiff(labels, x_min_3067_validation, y_min_3067, x_max_3067_validation, y_max_3067, labels_validation)
# Test
save_bbox_to_geotiff(labels, x_min_3067_test, y_min_3067, x_max_3067_test, y_max_3067, labels_test)

## Data image processing

1. Find data using Paituli STAC catalogue, separately for 2 dates.

In [ ]:
sentinel2_1 = fetch_stac_layer_with_bbox(sentinel2_stac_id, sentinel2_assets, sentinel2_time1, bbox_3067, data_epsg)
sentinel2_1

In [ ]:
sentinel2_2 = fetch_stac_layer_with_bbox(sentinel2_stac_id, sentinel2_assets, sentinel2_time2, bbox_3067, data_epsg)
sentinel2_2

Join data from both dates to one Xarray DataArray.

In [ ]:
sentinel2 = xarray.concat([sentinel2_1, sentinel2_2], dim="time")
sentinel2

Normalize the data values, the reflectance values are originally between 0 and 1, but for storage Sentinel-2 data has been multiplied with 10 000, so here the normalization is simple division with 10 000. For other data sources, this step is likely different.

In [ ]:
sentinel2 = sentinel2 / 10000.0

Plot small part of both scens as RGB images.

In [ ]:
cube_2021_rgb = sentinel2.isel(x=slice(1000,2000), y=slice(2000,3000)).sel(band=['b04', 'b03', 'b02'])
cube_2021_rgb.plot.imshow(row="time", rgb="band", robust=True, size=10)

Stack different data from different dates just as bands for saving it to a file.

In [ ]:
stacked = sentinel2.stack(z=("band", "time"))
stacked

Change the axis order for rasterio and add coordinate system info as rasterio wants to have it.

In [ ]:
final = stacked.transpose('z', 'y', 'x')
final.rio.write_crs("epsg:3067", inplace=True)
final

Save image data to a file.

In [ ]:
# Deep learning, model training
save_bbox_to_geotiff(final, x_min_3067_deep, y_min_3067, x_max_3067_deep, y_max_3067, data_deep)
# Shallow learning, model training
save_bbox_to_geotiff(final, x_min_3067_shallow, y_min_3067_shallow, x_max_3067_shallow, y_max_3067_shallow, data_shallow)
# Validation
save_bbox_to_geotiff(final, x_min_3067_validation, y_min_3067, x_max_3067_validation, y_max_3067, data_validation)
# Test
save_bbox_to_geotiff(final, x_min_3067_test, y_min_3067, x_max_3067_test, y_max_3067, data_test)

Double-check that all is fine. Pay attention to coordinate system, pixel size and data origin.

In [ ]:
!gdalinfo {data_test}

In [ ]:
!gdalinfo {labels_test} 

The data preparations for classification exercises are now ready, check `../data/raster` folder, that you have 8 .tif files there. Therea are also 8 .tif.aux.xml, which GDAL creates automatically with some metadata of the raster files.